In [3]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Dense, BatchNormalization, Dropout, Flatten, Activation, Lambda, Input
from keras.layers.convolutional import ZeroPadding2D
from keras.models import Sequential
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.layers.core import Lambda
from keras import initializers
from keras.utils import get_file
import tensorflow as tf
import numpy as np
import os

Using TensorFlow backend.


In [4]:
import collections

path = '/work/04381/ymarathe/maverick/yearbook/keras_yearbook/'
data_path = '/work/04381/ymarathe/maverick/yearbook/'

f = open(data_path + 'yearbook_train.txt', 'r')

freq = {};
fnamemap = {}
year2idx = {}
idx = 0
normal_const = 0;

for line in f:
    line = line.rstrip()
    image, year = line.split("\t")
    fnamemap[image] = year;
    if year in freq:
        freq[year] += 1
    else:
        freq[year] = 1

normal_const = np.sum(freq.values())
for key in freq:
    freq[key] = freq[key]/float(normal_const);
    
sorted_freq = collections.OrderedDict(sorted(freq.items()))

class_weights_train = {}
idx2year = {}
year2idx = {}

for idx, key in enumerate(sorted_freq):
    class_weights_train[idx] = sorted_freq[key]
    idx2year[idx] = key
    year2idx[key] = idx

In [5]:
f = open(data_path + 'yearbook_train.txt', 'r')

fnameToGenderTr = {}

for line in f:
    line = line.rstrip()
    image, year = line.split("\t")
    gender, imname = image.split("/")
    if gender is 'M':
        encodeGender = 1
    elif gender is 'F':
        encodeGender = 0
    fnameToGenderTr[image] = encodeGender

f = open(data_path + 'yearbook_valid.txt', 'r')

fnameToGenderVd = {}

for line in f:
    line = line.rstrip()
    image, year = line.split("\t")
    gender, imname = image.split("/")
    if gender is 'M':
        encodeGender = 1
    elif gender is 'F':
        encodeGender = 0
    fnameToGenderVd[image] = encodeGender

In [6]:
def gen_batches(path, gen = ImageDataGenerator(), shuffle=True, class_mode="categorical", batch_size=32, 
                target_size=(171, 186)):
    return gen.flow_from_directory(path, shuffle=shuffle, batch_size=batch_size, target_size=target_size, 
                                   class_mode=class_mode)

In [7]:
def vgg19_conv_layers_sequential():
    model = Sequential()
    
    model.add(ZeroPadding2D((1, 1), input_shape=(171, 186, 3)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(64, (3, 3), activation='relu'))

    model.add(MaxPooling2D((2, 2), strides = (2, 2)))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides = (2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides = (2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides = (2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides = (2, 2)))
    
    vgg_pretrain_weights = get_file('vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                    "https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5",
                                    cache_subdir='models')
    
    model.load_weights(vgg_pretrain_weights)
    
    return model

def vgg19_full_sequential():
    model = Sequential()
    
    model.add(ZeroPadding2D((1, 1), input_shape=(224, 224, 3)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(64, (3, 3), activation='relu'))

    model.add(MaxPooling2D((2, 2), strides = (2, 2)))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides = (2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides = (2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides = (2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides = (2, 2)))
    
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(1000, activation='softmax'))
    
    vgg_pretrain_weights = get_file('vgg19_weights_tf_dim_ordering_tf_kernels.h5',
                                                    'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels.h5',
                                                    cache_subdir='models',
                                                    file_hash='cbe5617147190e668d6c5d5026f83318')
    
    model.load_weights(vgg_pretrain_weights)
    
    return model

In [8]:
def vgg19_conv_functional(img_input):
    
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)
    
    return x

In [9]:
import math
import os
import numpy as np
from keras.preprocessing.image import Iterator
from keras.preprocessing.image import img_to_array, load_img
from keras.preprocessing.image import apply_transform, transform_matrix_offset_center
import keras.backend as K
from keras.preprocessing.image import random_rotation, random_shear, random_shift, random_zoom
from skimage import exposure
data_path = '/work/04381/ymarathe/maverick/yearbook/'

class ClassDataGen:
    def __init__(self, directory, map_file, 
                 target_size = (171, 186, 3), 
                 class_weights_train = None, 
                 multi_output=False, 
                 do_augmentation=True, 
                 samplewise_center = True,
                 samplewise_std_deviation = True,
                 year2idx = None
                ):
        self.directory = directory
        self.map_file = map_file
        self.filenames = []
        self.map = {}
        self.fnameToGender = {}
        self.target_size = target_size
        self.populate_filenames()
        self.populate_mapping()
        self.regressIter = None
        self.steps = 0
        self.samplewise_center = samplewise_center
        self.samplewise_std_deviation = samplewise_std_deviation
        self.height_shift_range = 0.2
        self.width_shift_range = 0.2
        self.max_rotation = 45
        self.shear = 0.785398
        self.zoom_range = (0.5, 0.5)
        self.do_augmentation = do_augmentation
        self.class_weights_train = class_weights_train
        self.equalizehist = False
        self.multi_output = multi_output
        self.lastN = []
        self.year2idx = year2idx;
        
    def _recursive_list(self, subpath):
        return sorted(
            os.walk(subpath, followlinks=False), key=lambda tpl: tpl[0])
    
    def populate_mapping(self):
        f = open(self.map_file, 'r')

        for line in f:
            line = line.rstrip()
            image, year = line.split("\t")
            gender, imfilename = image.split("/")
            if gender is 'M':
                encodeGender = 1
            elif gender is 'F':
                encodeGender = 0
            self.fnameToGender[image] = encodeGender
            self.map[image] = year
            
    def populate_filenames(self):
        base_dir = self.directory
        for root, _, files in self._recursive_list(base_dir):
            for fname in files:
                if fname.lower().endswith('.' + 'png'):
                    self.filenames.append(os.path.relpath(os.path.join(root, fname), base_dir))
                    
    def preprocess(self, x):
        if self.equalizehist:
            x = exposure.equalize_hist(x)
            
        return x
            
    def augment_data(self, x):
        
        x = random_shift(x, self.width_shift_range, self.height_shift_range, 
                         row_axis=0, col_axis = 1, channel_axis = 2)
        x = random_rotation(x, self.max_rotation, 
                            row_axis = 0, col_axis = 1, channel_axis = 2)
        x = random_shear(x, self.shear, row_axis = 0, col_axis = 1, channel_axis = 2)
        x = random_zoom(x, self.zoom_range, row_axis = 0, col_axis = 1, channel_axis = 2)
        
        return x
            
    def flow_from_directory(self, batch_size = 32, shuffle = True, seed = 42):
        
        self.regressIter = Iterator(len(self.filenames), batch_size = batch_size, shuffle = shuffle, seed = seed)
        
        if self.do_augmentation:
            factor = 3
        else:
            factor = 1
        
        self.steps = math.ceil(len(self.filenames)/batch_size) * factor
        
        return self
    
    def oneHotEncodeYear(self, year):
        integerEncoded = self.year2idx[year]
        oneHotVec = [0 for _ in range(len(self.year2idx))]
        oneHotVec[integerEncoded] = 1
        return oneHotVec
    
    def oneHotEncodeGender(self, gender):
        oneHotVec = [0, 0]
        oneHotVec[gender] = 1;
        return oneHotVec
    
    def next(self, *args, **kwargs):
           
        self.lastN = []
        
        idx_array, cur_idx, bs = next(self.regressIter.index_generator)
        
        batch_x = np.zeros(tuple([len(idx_array)] + list(self.target_size)), dtype=K.floatx())
        
        batch_y = np.zeros(tuple([len(idx_array)] + list([len(self.year2idx)])), dtype=K.floatx())
        
        if self.multi_output:
            batch_y_gender = np.zeros(tuple([len(idx_array)] + list([2])), dtype=K.floatx())

        if self.class_weights_train is not None:
            sample_weights = np.ones(tuple([len(idx_array)]), dtype=K.floatx())
        
        for i, j in enumerate(idx_array):
            fname = self.filenames[j]
            self.lastN.append(fname)
            img = load_img(
                  os.path.join(self.directory, fname),
                  grayscale = True,
                  target_size= self.target_size)
            x = np.array(img_to_array(img, data_format='channels_last'))
            x = self.preprocess(x)
            batch_x[i] = x
            batch_y[i, :] = self.oneHotEncodeYear(self.map[fname])
            
            if self.multi_output:
                batch_y_gender[i, :] = self.oneHotEncodeGender(self.fnameToGender[fname])
            
            if self.class_weights_train is not None:
                if self.multi_output:
                    sample_weights[i] = self.class_weights_train[self.map[fname]]
                else:
                    sample_weights[i] = self.class_weights_train[self.map[fname]]
        
        if self.samplewise_center:
            for x in batch_x:
                x -= np.mean(x)
        
        if self.samplewise_std_deviation:
            for x in batch_x:
                x /= np.std(x)
        
        if self.do_augmentation:
            for x in batch_x:
                x = self.augment_data(x)
        
        if self.multi_output:
            if self.class_weights_train is not None:
                return batch_x, {'out_year' : batch_y, 'out_gender': batch_y_gender}, {'out_year' : sample_weights, 'out_gender' : sample_weights} 
            else:
                return batch_x, {'out_year' : batch_y, 'out_gender': batch_y_gender}
        else:    
            if self.class_weights_train is not None:
                return (batch_x, batch_y, sample_weights)
            else:
                return (batch_x, batch_y)

In [10]:
train = ClassDataGen(data_path + 'train',
                       data_path + 'yearbook_train.txt', 
                       class_weights_train = sorted_freq, 
                       multi_output=True,
                       do_augmentation=True,
                       year2idx = year2idx
                       )

valid = ClassDataGen(data_path + 'valid',
                       data_path + 'yearbook_valid.txt',
                       class_weights_train = sorted_freq,
                       do_augmentation=False,
                       multi_output=True,
                       year2idx = year2idx
                      )

train = train.flow_from_directory()
valid = valid.flow_from_directory(shuffle=False)

In [11]:
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras.layers import Input
from keras.models import Model
from keras.layers import merge

img_input = Input(shape=(171, 186, 3))
x = vgg19_conv_functional(img_input)
x = Flatten()(x)
x = Dense(4096, kernel_initializer='glorot_normal', bias_initializer=keras.initializers.Ones())(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
out_year = Dense(104, activation='softmax', name = 'out_year')(x)
out_gender = Dense(2, activation='softmax', name = 'out_gender')(x)

def customMetric(y_true, y_pred):
    return K.mean(K.abs(int(idx2year[np.nonzero(y_true)[0][0]]) - int(idx2year[np.nonzero(y_pred)[0][0]])))

model = Model([img_input], [out_year, out_gender])

lr = 1e-4

model.compile(Adam(lr=lr), loss=['categorical_crossentropy', 'categorical_crossentropy'], 
              metrics=['accuracy'],
              loss_weights = [0.7, 0.3]
            )

def lr_schedule(epoch):
    return lr * (0.1 ** int(epoch / 10))

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 171, 186, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Conv2D)            (None, 171, 186, 64)  1792        input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv2 (Conv2D)            (None, 171, 186, 64)  36928       block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, 85, 93, 64)    0           block1_conv2[0][0]               
___________________________________________________________________________________________

with tf.device('/gpu:0'):
    print ("Saving result in exp45.h5")
    model.fit_generator(train, steps_per_epoch = train.steps, epochs = 5,                                
                               validation_data = valid, 
                               validation_steps = valid.steps,
                               callbacks=[LearningRateScheduler(lr_schedule),
                               ModelCheckpoint(path + 'weights/exp45.h5', save_best_only=True)])

In [12]:
model.load_weights(path + 'weights/exp44.h5')

In [13]:
valid = valid.flow_from_directory(shuffle = False, batch_size = 1)

In [ ]:
y_true = []
y_pred = []
mae = 0
N = len(valid.filenames)
for x in range(len(valid.filenames)):
    batch_x, batch_y, sample_weights = next(valid)
    true_year = int(idx2year[np.argmax(batch_y['out_year'])])
    y_pred_prob = model.predict(batch_x)
    pred_year = int(idx2year[np.argmax(y_pred_prob[0])])
    y_true.append(true_year)
    y_pred.append(pred_year)
    mae += abs(true_year - pred_year)

print mae/N

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
np.save(path + 'weights/exp44_cm.npy', confusion_matrix(y_true, y_pred))